In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Code for plotting ACF,PACF and Q-Q plots. Taken from BlackArbs LLC website
import os
import sys

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs


#import matplotlib.pyplot as plt
import matplotlib as mpl

def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
#Load Data
df = pd.read_excel('2018_2020_NiftyIV_v2.xlsx')

In [ ]:
#Create a label to identify the transition from one expiry to another. This piece of code identifies the change in
#days to expiry and creates a abel for it.
df['Label'] = 0
df['Label'] = np.where(df['Days_To_Expiy']>=df['Days_To_Expiy'].shift(-1),0,1)
#df.head(5)

In [ ]:
#Create a list of dataframes that group the data by dates
dfDailyList = []
for group in df.groupby(df['Date/Time'].dt.date):
    dfDailyList.append(group[1])

In [ ]:
#Slice the main data by expiry, create a list of dataframes for different expiries
index_list = df[df['Label']==1].index.tolist()
dfExpiryList = []
for i in range(0,len(index_list)):
    if(i==0):
        dfExpiryList.append(df.iloc[:index_list[i]+1,:])
    else:
        dfExpiryList.append(df.iloc[index_list[i-1]+1:index_list[i]+1,:])

In [ ]:
#Create an OHLC data for price and volatility on day basis
from datetime import datetime
column_names = ['Date/Time','vol_Open','vol_High','vol_Low','vol_Close','price_Open','price_High','price_Low','price_Close','Label']
dfDailyOHLCList = pd.DataFrame(columns=column_names)
for dftemp in dfDailyList:
        dte = dftemp['Date/Time'].iloc[0].date()
        vol_Open = dftemp['Avg_IV_ATM'].iloc[0]
        vol_Close = dftemp['Avg_IV_ATM'].iloc[-1]
        vol_High = dftemp['Avg_IV_ATM'].max()
        vol_Low = dftemp['Avg_IV_ATM'].min()
        price_Open = dftemp['Underlying_Fut'].iloc[0]
        price_Close = dftemp['Underlying_Fut'].iloc[-1]
        price_High = dftemp['Underlying_Fut'].max()
        price_Low = dftemp['Underlying_Fut'].min()
        label = dftemp['Label'].iloc[-1]
        dfDailyOHLCList = dfDailyOHLCList.append({'Date/Time':dte,'vol_Open':vol_Open,'vol_High':vol_High,'vol_Low':vol_Low,'vol_Close':vol_Close,'price_Open':price_Open,'price_High':price_High,'price_Low':price_Low,'price_Close':price_Close,'Label':label},ignore_index=True)
        

In [ ]:
#Extract data at 3.15 pm verydat
column_names = ['Date','vol','price']
dfDailyEODList = pd.DataFrame(columns=column_names)
my_time_string = "15:15:00"
my_time = datetime.strptime(my_time_string, "%H:%M:%S")

i=0
for dftemp in dfDailyList:
        dte = dftemp['Date/Time'].iloc[0].date()
        vol = dftemp[dftemp['Date/Time'][:].dt.time == my_time.time()]['Avg_IV_ATM'].to_numpy()
        price = dftemp[dftemp['Date/Time'][:].dt.time == my_time.time()]['Underlying_Fut'].to_numpy()
        #label = dftemp[dftemp['Date/Time'][:].dt.time == my_time.time()]['Label']
        if(vol.size == 0):
            continue
        else:
            vol1=vol[0,]
            price1=price[0,]
        i=i+1
        dfDailyEODList = dfDailyEODList.append({'Date':dte,'vol':vol1,'price':price1},ignore_index=True)

In [ ]:
#A sample GARCH problem, taken from BalckArbs LLC
#This works spectacularly
from arch import arch_model
a0 = 0.2
a1 = 0.5
b1 = 0.3

n = 10000
w = np.random.normal(size=n)
eps = np.zeros_like(w)
sigsq = np.zeros_like(w)

for i in range(1, n):
    sigsq[i] = a0 + a1*(eps[i-1]**2) + b1*sigsq[i-1]
    eps[i] = w[i] * np.sqrt(sigsq[i])

_ = tsplot(eps)
am = arch_model(eps)
res = am.fit(update_freq=5)
res.summary()

In [ ]:
#Volatility calculation based on given price data, WORK IN porgress
df['Return'] = np.log(df['Underlying_Fut']/df['Underlying_Fut'].shift(1))
df['Cum Return'] = df['Return'].cumsum()
df['Return_Vol'] = np.sqrt(df['Return'].apply(lambda x: x*x))*np.sqrt(252)*100
layout = (1, 1)
ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
df['Return_Vol'].plot(ax=ts_ax)
df['Avg_IV_ATM'].plot(ax=ts_ax)
plt.show()

In [ ]:
#Plot of volatilities @ 3.15 pm data
tsplot(dfDailyEODList['vol'],lags=30)

In [ ]:
#GARCH on EOD data, note that the beta value shows fit is imperfect
am = arch_model(dfDailyEODList['vol'],p=1,q=1)
res = am.fit(update_freq=5)
res.summary()

In [ ]:
#Volatility calculations - Work in progress
dfDailyOHLCList['Return'] = np.log(dfDailyOHLCList['price_Close']/dfDailyOHLCList['price_Close'].shift(1))
dfDailyOHLCList['Parkinson_Return'] = np.log(dfDailyOHLCList['price_High']/dfDailyOHLCList['price_Low'])**2
dfDailyOHLCList['Cum Return'] = dfDailyOHLCList['Return'].cumsum()
length_return = np.arange(1,len(dfDailyOHLCList['Return'])+1)
length_parkinson = np.arange(1,len(dfDailyOHLCList['Parkinson_Return'])+1)
dfDailyOHLCList['Return_Vol'] = np.sqrt(dfDailyOHLCList['Return'].apply(lambda x: x*x)).divide(length_return)*np.sqrt(252)*100
dfDailyOHLCList['Parkinson_Vol'] = np.sqrt(dfDailyOHLCList['Parkinson_Return'].cumsum()/4/(len(dfDailyOHLCList['Parkinson_Return']))/np.log(2))*100
layout = (1, 1)
ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
dfDailyOHLCList['Return_Vol'].plot(ax=ts_ax,color='r')
(dfDailyOHLCList['vol_Close']/100).plot(ax=ts_ax,color='b')
dfDailyOHLCList['Parkinson_Vol'].plot(ax=ts_ax,color='c')
plt.show()

In [ ]:
_ = tsplot(dfDailyOHLCList['vol_Close'])

In [ ]:
#GARCH on the close data from OHLC data
am = arch_model(dfDailyOHLCList['vol_Close'][:100],p=1,q=1)
res = am.fit(update_freq=5)
print(res.summary())
res.plot()

In [ ]:
#GARCH on the first daily OHLC data
dftemp = dfDailyList[0]
tsplot(dftemp['Avg_IV_ATM'],lags=30)

In [ ]:
#GARCH on the first daily OHLC data, note it did not converge
am = arch_model(dftemp['Avg_IV_ATM'],p=1,q=1)
res = am.fit(update_freq=5)
print(res.summary())
res.plot()

In [ ]:
#GARCH on the expiry data
dftemp = dfExpiryList[0]
tsplot(dftemp['Avg_IV_ATM'],lags=30)

In [ ]:
#GARCH on the first daily OHLC data, only this model looks a bit better
am = arch_model(dftemp['Avg_IV_ATM'],p=1,q=1)
res = am.fit(update_freq=5)
print(res.summary())
res.plot()